check if headers are there, if they aren't add them

In [ ]:
directory = '/Users/adamhunter/Desktop/WedgeZipOfZips/' #kept this out of my folder that was open in cursor as it was leading to lag

unzip, standardize delimiters, null placeholders, write things back with quotes around everything to help deal with a few items that have , in them like Fruit, Granola, and Nuts or something

In [ ]:
import os
import pandas as pd
import csv

import zipfile
from tqdm.notebook import tqdm

# Delete all CSV files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        os.remove(os.path.join(directory, filename))

# Unzip all ZIP files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.zip'):
        with zipfile.ZipFile(os.path.join(directory, filename), 'r') as zip_ref:
            zip_ref.extractall(directory)

# Loop through each file in the directory
for filename in tqdm(os.listdir(directory)):
    # Only process .csv files
    if filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)
        
        # Determine the delimiter by looking for a semicolon in first two lines
        with open(file_path, 'r') as file:
            first_two_lines = [next(file) for x in range(2)]
            delimiter = ';' if any(';' in line for line in first_two_lines) else ','
        
        # Read the file using the determined delimiter
        df = pd.read_csv(file_path, delimiter=delimiter)
        
        # Replace all occurrences of '\N' or 'NULL' with '' for consistency, in hindsight this would have been easier if i replaced with None
        df.replace(['\\N', 'NULL', '\\\\N'], '', inplace=True)
        
        # Write the file out again with quotes around each value, this handles product descriptions that contain delimiters
        df.to_csv(file_path, index=False, quoting=csv.QUOTE_ALL)


Add headers if they are missing

In [ ]:
# Define the standard headers
standard_headers = ["datetime", "register_no", "emp_no", "trans_no", "upc", "description", "trans_type", "trans_subtype", "trans_status", "department", "quantity", "Scale", "cost", "unitPrice", "total", "regPrice", "altPrice", "tax", "taxexempt", "foodstamp", "wicable", "discount", "memDiscount", "discountable", "discounttype", "voided", "percentDiscount", "ItemQtty", "volDiscType", "volume", "VolSpecial", "mixMatch", "matched", "memType", "staff", "numflag", "itemstatus", "tenderstatus", "charflag", "varflag", "batchHeaderID", "local", "organic", "display", "receipt", "card_no", "store", "branch", "match_id", "trans_id"]


# Loop through each file in the directory
for filename in tqdm(os.listdir(directory)):
    # Only process .csv files
    if filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)
        
        # Read the file
        with open(file_path, 'r') as file:
            reader = csv.reader(file)
            headers = next(reader, None)
        
        # Check if headers match the standard headers
        if headers != standard_headers:
            # If not, read the original content into a DataFrame
            with open(file_path, 'r') as file:
                df = pd.read_csv(file)  # Skip the first line

            # Write the standard headers and the original content to the file, quotes help if delimiters happen to be in product descriptions
            df.to_csv(file_path, header=standard_headers, index=False, quoting=csv.QUOTE_ALL)

In [ ]:
# Define the standard headers and their corresponding data types
data_types = {
    "datetime": 'datetime64[ns]', 
    "register_no": 'float64', 
    "emp_no": 'float64', 
    "trans_no": 'float64', 
    "upc": 'str', 
    "description": 'str', 
    "trans_type": 'str', 
    "trans_subtype": 'str', 
    "trans_status": 'str', 
    "department": 'float64', 
    "quantity": 'float64', 
    "Scale": 'float64', 
    "cost": 'float64', 
    "unitPrice": 'float64', 
    "total": 'float64', 
    "regPrice": 'float64', 
    "altPrice": 'float64', 
    "tax": 'float64', 
    "taxexempt": 'float64', 
    "foodstamp": 'float64', 
    "wicable": 'float64', 
    "discount": 'float64', 
    "memDiscount": 'float64', 
    "discountable": 'float64', 
    "discounttype": 'float64', 
    "voided": 'float64', 
    "percentDiscount": 'float64', 
    "ItemQtty": 'float64', 
    "volDiscType": 'float64', 
    "volume": 'float64', 
    "VolSpecial": 'float64', 
    "mixMatch": 'float64', 
    "matched": 'float64', 
    "memType": 'bool', 
    "staff": 'bool', 
    "numflag": 'float64', 
    "itemstatus": 'float64', 
    "tenderstatus": 'float64', 
    "charflag": 'str', 
    "varflag": 'float64', 
    "batchHeaderID": 'bool', 
    "local": 'float64', 
    "organic": 'float64', 
    "display": 'bool', 
    "receipt": 'float64', 
    "card_no": 'float64', 
    "store": 'float64', 
    "branch": 'float64', 
    "match_id": 'float64',
    "trans_id": 'float64'
}

In hindsight it is possible that some of the complexity in the next cell is because of replacing 'NULL' and '\N' with '' instead of replacing with None.

In [ ]:
import numpy as np
import os
import pandas as pd
import csv

for filename in tqdm(os.listdir(directory)):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)
        
        df = pd.read_csv(file_path) #throws warnings about mixed dtypes

        for col, dtype in data_types.items():
            if dtype == 'float64':
                df[col] = pd.to_numeric(df[col], errors='coerce') #first step deals with nan
                df[col] = df[col].astype(dtype) #second step makes things that should be int into float. trying this directly didn't deal with nan properly.
            else:
                df[col] = df[col].astype(dtype) #if intended dtype isn't float, convert to bool/str as appropriate
        
        df = df.replace({'nan': None}) #certain string columns were having null turned into 'nan' by pandas? which then turned the column into strings?

        # Write the cleaned DataFrame to a new CSV file
        cleaned_file_path = os.path.join(directory, f"cleaned_{filename}")
        df.to_csv(cleaned_file_path, index=False)

In [ ]:
import pandas as pd
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
  'adamh-wedge-project-6f59b14d0763.json')

uploading files, had an issue with datetimes not saving properly in csv so changing in df just before upload

thread pool approach cut upload time in half

In [ ]:
from concurrent.futures import ThreadPoolExecutor

def upload_to_gbq(filename):
    if filename.startswith('cleaned_') and filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)
        
        # Load the cleaned CSV file into a pandas DataFrame
        df = pd.read_csv(file_path)
        df['datetime'] = pd.to_datetime(df['datetime'])
        # Extract table name from the filename
        table_name = filename[8:-4]  # Remove the 'cleaned_' prefix and .csv extension

        table_id = f'adamh-wedge-project.wedge_transactions.{table_name}'
        
        # Upload the DataFrame to GBQ
        df.to_gbq(table_id, credentials=credentials, if_exists='replace')

with ThreadPoolExecutor(max_workers=4) as executor:
    list(tqdm(executor.map(upload_to_gbq, os.listdir(directory)), total=len(os.listdir(directory))))